## Testing Playground

You've probably noticed this project lacks a unit testing set-up. Honestly, I found unit testing tricky for this and felt visual inspections of outputs, like markdowns or visualizations, were generally more trustworthy.

But I get the worry of accidentally breaking things when contributing code. So, I made this notebook to help with that. It's a work in progress, aimed at letting you easily see specific outputs of interest for smoother development.

Previously, my basic testing meant using a `test.py` file to generate a few markdowns and then checking them manually. Or, for a deeper look, I'd run the `main.py` and wait a while to see everything, which isn't quick on my laptop.

This notebook aims to streamline that process, letting you test and inspect targeted parts of the output without the fear of breaking things.

Good examples to inspect include:

- Conversations with multiple branches, e.g., prompt edits, re-submissions, to see message order and dynamic linking like `[child ⬇️](#id-string)`.
- Chats involving code interpreter.
- Interactions with plugins and their textual outputs.
- Chats with images, links, etc.
- Sessions spanning different models, like starting with gpt-4 and switching to gpt-3.5.
- Graphs, word clouds, and so on.

### Go ahead and add to the cells below...

Currently juggling school, so my time's limited. Would appreciate any contributions to help grow this project. Thanks!

#### Example ...

In [ ]:
"""You need to RESTART the kernel to see changes you made in the modules imported below."""

import json
from pathlib import Path
from random import randint


# You need to RESTART the kernel to see changes you made in the modules imported below.
from models.conversation import Conversation

# create a 'data' folder in the root directory to store your personal data to test on

data_path = Path("data")
data_path.mkdir(exist_ok=True)

# assuming you put the 'conversations.json' file in the 'data' folder

conversations_path = data_path / "conversations.json"

with open(conversations_path, "r", encoding="utf-8") as f:
    conversations = json.load(f)

# create an 'output' folder in the root directory to store the output files

output_path = Path("output")
output_path.mkdir(exist_ok=True)


def clear_output_folder():
    for file in output_path.glob("*"):
        file.unlink()


conversation_list = [Conversation(**c) for c in conversations]

In [ ]:
# selecting five conversations with the highest leaf count

avg_leaf_count = sum(c.leaf_count for c in conversation_list) / len(conversation_list)

median_leaf_count = sorted(c.leaf_count for c in conversation_list)[
    len(conversation_list) // 2
]

max_leaf_count = max(c.leaf_count for c in conversation_list)

print(f"Average leaf count: {avg_leaf_count}")

print(f"Median leaf count: {median_leaf_count}")

print(f"Max leaf count: {max_leaf_count}")

print()

convos_sorted_by_leaf_count = sorted(conversation_list, key=lambda c: c.leaf_count)

for convo in convos_sorted_by_leaf_count[-5:]:
    print(f"id: {convo.id}")
    print(f"title: {convo.title}")
    print(f"leaf count: {convo.leaf_count}")
    convo.save_to_dir(output_path)
    print(f"saved to '{output_path}/'")
    print()

In [ ]:
clear_output_folder()

In [ ]:
# selecting five conversations with the highest message count

avg_message_count = sum(c.message_count for c in conversation_list) / len(
    conversation_list
)

median_message_count = sorted(c.message_count for c in conversation_list)[
    len(conversation_list) // 2
]

max_message_count = max(c.message_count for c in conversation_list)

print(f"Average message count: {avg_message_count}")

print(f"Median message count: {median_message_count}")

print(f"Max message count: {max_message_count}")

print()

convos_sorted_by_message_count = sorted(
    conversation_list, key=lambda c: c.message_count
)

for convo in convos_sorted_by_message_count[-5:]:
    print(f"id: {convo.id}")
    print(f"title: {convo.title}")
    print(f"message count: {convo.message_count}")
    convo.save_to_dir(output_path)
    print(f"saved to '{output_path}/'")
    print()

In [ ]:
clear_output_folder()

In [ ]:
# selecting five conversations with the most content types

avg_content_type_count = sum(len(c.content_types) for c in conversation_list) / len(
    conversation_list
)

median_content_type_count = sorted(len(c.content_types) for c in conversation_list)[
    len(conversation_list) // 2
]

max_content_type_count = max(len(c.content_types) for c in conversation_list)

print(f"Average content type count: {avg_content_type_count}")

print(f"Median content type count: {median_content_type_count}")

print(f"Max content type count: {max_content_type_count}")

print()

convos_sorted_by_content_type_count = sorted(
    conversation_list, key=lambda c: len(c.content_types)
)

for convo in convos_sorted_by_content_type_count[-5:]:
    print(f"id: {convo.id}")
    print(f"title: {convo.title}")
    print(f"content types: {convo.content_types}")
    convo.save_to_dir(output_path)
    print(f"saved to '{output_path}/'")
    print()

In [ ]:
clear_output_folder()